In [1]:
import mitsuba as mi
import drjit as dr
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random 
import imageio
import cv2

In [2]:
scene_path = os.path.join('', 'scenes') 
results_spatio_path = os.path.join('','results_spatio') 
mi.set_variant('scalar_rgb')
texture_path = os.path.join(scene_path,'textures')

tokki = os.path.join(scene_path,'meshes/bunny.ply')


In [3]:
cam1 = mi.load_dict({
'type': 'perspective',
'fov': 45, #24.6 mm
'to_world': mi.ScalarTransform4f.look_at(
    origin=[-0.03, 0, -0.7],  # increase the z-coordinate to move the camera farther away
    target=[-0.03, 1, -15],
    up=[0, 0, 1]
),
'film2': {
    'type': 'hdrfilm',
    'width': 360,
    'height': 360,
}
# 'distortion': [0, 0, 0, 0, 0]
})

In [8]:
import cgi, os
files = os.listdir('dir/graycode_pattern/')

for f in files:
    s = "dir/graycode_pattern/"
    s = s + f
    scene1 = mi.load_dict({
        'type':'scene',
        'integrator': {'type': 'direct'},
        'light': {
            'type': 'projector',
            'irradiance': {
                'type': 'bitmap',
                'filename': s, 
            },
            'fov': 45,
            'to_world': mi.ScalarTransform4f.look_at(
                origin=[-0.03, 0, -0.7],
                target=[-0.03, 1, -15],
                up=[0, 0, 1]
            )
            },
        'tok':{
            'type': 'ply',
            'filename': tokki,
            'to_world': mi.ScalarTransform4f.translate([0, 0, -1]),
            'bsdf': {
                'type': 'diffuse',
                # 'reflectance': {'type': 'rgb', 'value': [0.1, 0.2, 0.3]},
            }
        }
    })

    image = mi.render(scene=scene1, sensor=cam1, spp=64)
    image = np.clip(image * 1000, 0, 1000).astype(np.uint8)
    st = "./results/resultfar_" + f
    # Save the image as a PNG file using imageio
    imageio.imwrite(st, image)


In [8]:
help(cv2.Rodrigues)

Help on built-in function Rodrigues:

Rodrigues(...)
    Rodrigues(src[, dst[, jacobian]]) -> dst, jacobian
    .   @brief Converts a rotation matrix to a rotation vector or vice versa.
    .   
    .   @param src Input rotation vector (3x1 or 1x3) or rotation matrix (3x3).
    .   @param dst Output rotation matrix (3x3) or rotation vector (3x1 or 1x3), respectively.
    .   @param jacobian Optional output Jacobian matrix, 3x9 or 9x3, which is a matrix of partial
    .   derivatives of the output array components with respect to the input array components.
    .   
    .   \f[\begin{array}{l} \theta \leftarrow norm(r) \\ r  \leftarrow r/ \theta \\ R =  \cos(\theta) I + (1- \cos{\theta} ) r r^T +  \sin(\theta) \vecthreethree{0}{-r_z}{r_y}{r_z}{0}{-r_x}{-r_y}{r_x}{0} \end{array}\f]
    .   
    .   Inverse transformation can be also done easily, since
    .   
    .   \f[\sin ( \theta ) \vecthreethree{0}{-r_z}{r_y}{r_z}{0}{-r_x}{-r_y}{r_x}{0} = \frac{R - R^T}{2}\f]
    .   
    .   A rot

In [40]:
# Get the rotation matrix from the camera transformation matrix
cam_transform = mi.ScalarTransform4f.look_at(
    origin=[0, 0, -1.1],
    target=[0, 1, -15],
    up=[0, 0, 1]
)
cam_to_world = np.array(cam_transform.matrix)
cam_rot_vec, _ = cv2.Rodrigues(cam_to_world[:3, :3])
cam_rot_mat = np.array(cam_rot_vec)

# Define the proj_dist and proj_int matrices
proj_dist = np.zeros((4, 1))

proj =  mi.ScalarTransform4f.look_at(
            origin=[0, 0, -0.9],
            target=[0, 1, -15],
            up=[0, 0, 1]
        )

proj_int = np.array([[360/(2*np.tan(45*np.pi/360)) , 0, 360/2],
                     [0, 360/(2*np.tan(45*np.pi/360)), 360/2],
                     [0, 0, 1]])
# Compute the cam_proj_rmat matrix
# cam_proj_rmat = np.dot(proj_int, np.hstack((cam_rot_mat, -np.dot(cam_rot_mat, cam_to_world[:3, 3].reshape(-1,1)))))

cam_proj_rmat = np.dot(proj_int, cam_to_world[:3, :3])
cam_proj_rmat




ValueError: shapes (3,1) and (3,1) not aligned: 1 (dim 1) != 3 (dim 0)

ValueError: shapes (3,1) and (3,) not aligned: 1 (dim 1) != 3 (dim 0)

array([[-1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.9974221 ,  0.07175699,  0.        ],
       [ 0.        ,  0.07175699, -0.9974221 , -1.1       ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [34]:
proj_int = np.array([[360/(2*np.tan(45*np.pi/360)) , 0, 360/2, 0],
                     [0, 360/(2*np.tan(45*np.pi/360)), 360/2, 0],
                     [0, 0, 1, 0]])
proj_int

array([[434.55844123,   0.        , 180.        ,   0.        ],
       [  0.        , 434.55844123, 180.        ,   0.        ],
       [  0.        ,   0.        ,   1.        ,   0.        ]])